# Install System Dependencies
Use shell commands to install necessary system dependencies.


In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y make build-essential libssl-dev zlib1g-dev libbz2-dev \
    libreadline-dev libsqlite3-dev wget curl llvm libncurses5-dev libncursesw5-dev \
    xz-utils tk-dev libffi-dev liblzma-dev git lshw

# Install Python Packages
Use pip to install required Python packages.

In [ ]:
%%capture

import subprocess
import sys

# Define the requirements array
requirements = [
    "transformers",
    "langchain",
    "langchain-community",
    "langchain-ollama",
    "termcolor"
]

# Install the packages
for package in requirements:
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install & Setup Ollama
Use shell commands

In [3]:
import subprocess
from termcolor import cprint, colored

# Function to run shell commands
def run_shell_command(command: str, async_mode: bool = False):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if not async_mode:
        stdout, stderr = process.communicate()
        if process.returncode != 0:
            cprint(f"Error running command: {command}\n{stderr.decode('utf-8')}", "red")
        else:
            cprint(f"Command ran successfully: {command}\n{stdout.decode('utf-8')}", "green")
    return process
        
# Check if Ollama is installed
def is_ollama_installed() -> bool:
    try:
        subprocess.check_call(["ollama", "--version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return True
    except subprocess.CalledProcessError:
        return False
    except FileNotFoundError:
        return False

# Install Ollama if not already installed
if not is_ollama_installed():
    cprint("Installing Ollama...", "yellow")
    run_shell_command("curl -fsfL https://ollama.ai/install.sh | sh")
else:
    cprint("Ollama is already installed.", "green")

# Start Ollama serve asynchronously
# cprint("Starting Ollama serve...", "yellow")
# run_shell_command("ollama serve", async_mode=True)


Ollama is already installed.


# Define Constants and Import Libraries
Define constants like MODEL_NAME and import necessary libraries.

In [ ]:
# Import necessary libraries
from langchain_community.llms import Ollama

# Define the model name constant
# Define a list to map numbers to model names
model_list = [
    "deepseek-coder-v2:16b", # 8.9GB
    "llama3.2:1b" , #1.3GB
    "qwen2.5-coder:1.5b", # 986MB
]

def get_model_name(num: int) -> str:
    if 1 <= num <= len(model_list):
        return model_list[num - 1]
    else:
        return "deepseek-coder-v2:16b"  # Default model if the number is out of range

# Pull the selected model
selected_model = get_model_name(1)
cprint(f"Pulling model: {selected_model}", "yellow")
run_shell_command(f"ollama run {selected_model}")

# Initialize Ollama Instance
llm = Ollama(model=selected_model)
cprint("Selected model: " + colored(llm.model, "green"))

# Define Prompt Template
Define the prompt template used for generating summary documentation.

In [ ]:
prompt_template = '''
You are a software engineer tasked with generating summary comment documentation for a given code snippet. Follow these steps:

1. **Review the Summary Policy**: Understand the guidelines for summarizing code.
2. **Analyze the Code Content**: Identify the programming language and functionality of the provided code snippet.
3. **Generate Summary Documentation**: Create a concise summary that describes what the code does, its purpose, and any important details.

### Summary Policy
<SUMMARY_POLICY>
{$SUMMARY_POLICY}
</SUMMARY_POLICY>

### Code Content
<CODE_CONTENT>
{$CODE_CONTENT}
</CODE_CONTENT>

### Output Format
```[lang]
[code with summary]
```
- The entire code block is wrapped in triple backticks (`) to denote it as a code snippet.
- The [lang] specifies the programming language used for clarity and context.
- The [code with summary] section provides both the original code and its documentation.

### Example Outputs
<example1>
Input:
```py
def add(a, b):
    return a + b
```

Output:
```py
def add(a, b):
    """
    Adds two numbers.

    @param a: (int or float) The first number to add.
    @param b: (int or float) The second number to add.
    @return: (int or float) The sum of `a` and `b`.
    """
    return a + b
```
</example 1>

<example2>
Input:
```py
class Library:
    def __init__(self):
        self.books = {}

    def add_book(self, title, author):
        self.books[title] = author

    def checkout_book(self, title):
        if title in self.books:
            del self.books[title]
            return f"You have checked out '{title}'."
        else:
            return f"'{title}' is not available in the library."
```

Output:
```py
class Library:
    """
    A class to manage a collection of books in a library.
    """

    def __init__(self):
        """
        Initializes a new Library instance with an empty collection of books.
        """
        self.books = {}

    def add_book(self, title, author):
        """
        Adds a book to the library.

        @param title: (str) The title of the book to add.
        @param author: (str) The author of the book to add.
        """
        self.books[title] = author

    def checkout_book(self, title):
        """
        Checks out a book from the library.

        @param title: (str) The title of the book to check out.
        @return: (str) A message indicating whether the checkout was successful or if the book is not available.
        """
        if title in self.books:
            del self.books[title]
            return f"You have checked out '{title}'."
        else:
            return f"'{title}' is not available in the library."
```
</example2>
'''

# Predefined Code Examples
List predefined code examples that will be used for generating responses.

In [ ]:
# Predefined Code Examples

# List of predefined code examples that will be used for generating responses
code_examples = [
    """
def fibonacci(n):
    sequence = []
    a, b = 0, 1
    while a < n:
        sequence.append(a)
        a, b = b, a + b
    return sequence
    """,
    """
class Restaurant:
    def __init__(self, name, cuisine_type):
        self.name = name
        self.cuisine_type = cuisine_type
        self.menu = {}

    def add_dish(self, dish_name, price):
        self.menu[dish_name] = price

    def remove_dish(self, dish_name):
        if dish_name in self.menu:
            del self.menu[dish_name]

    def get_menu(self):
        return self.menu
    """,
    """
def merge_sorted_lists(list1, list2):
    merged_list = []
    i, j = 0, 0

    while i < len(list1) and j < len(list2):
        if list1[i] < list2[j]:
            merged_list.append(list1[i])
            i += 1
        else:
            merged_list.append(list2[j])
            j += 1

    merged_list.extend(list1[i:])
    merged_list.extend(list2[j:])

    return merged_list
    """
]

# Generate Response Function
Define a function to generate responses from the model using the prompt template.

In [ ]:
import time
from termcolor import cprint

summary_policy = "Summarize code clearly and concisely and functionality."

def generate_response(code_content: str, summary_policy: str) -> str:
    """Generate a response from the model using the prompt template."""
    prompt = prompt_template.replace("{$SUMMARY_POLICY}", summary_policy).replace("{$CODE_CONTENT}", code_content)
    cprint(f"actual_prompt:\n{prompt}", "magenta")
    
    start_time = time.time()
    response = llm(prompt)
    end_time = time.time()
    
    time_taken = end_time - start_time
    cprint(f"Time taken: {time_taken:.2f} seconds", "blue")
  
    return response
    
# Generate responses for predefined instructions
for code_content in code_examples:
    cprint("=" * 30, "cyan") # Colored separator
    cprint(f"Code Content:\n{code_content}", "yellow")
    output = generate_response(code_content, summary_policy)
    cprint(f"Response from model:\n{output}\n", "green")
    cprint("=" * 30, "cyan") # Colored separator